<img src='https://i.imgur.com/RDAD11M.png' width = '200' align = 'right'>

# 📝  Project1

In [1]:
import pandas as pd
import numpy as np
import re
from google.colab import files
import io
import matplotlib.pyplot as plt 
import random
import seaborn as sns

In [4]:
uploaded = files.upload()

Saving vgames2.csv to vgames2.csv


In [5]:
df = pd.read_csv('vgames2.csv',index_col=0)

In [12]:
df.head(3)

,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales
1,Candace Kane's Candy Factory,DS,2008.0,Action,Destineer,0.04,0,0,0
2,The Munchables,Wii,2009.0,Action,Namco Bandai Games,0.17,0,0,0.01
3,Otome wa Oanesama Boku ni Koi Shiteru Portable,PSP,2010.0,Adventure,Alchemist,0,0,0.02,0


Name : 게임의 이름입니다.

Platform : 게임이 지원되는 플랫폼의 이름입니다.

Year : 게임이 출시된 연도입니다.

Genre : 게임의 장르입니다.

Publisher : 게임을 제작한 회사입니다.

NA_Sales : 북미지역에서의 출고량입니다.

EU_Sales : 유럽지역에서의 출고량입니다.

JP_Sales : 일본지역에서의 출고량입니다.

Other_Sales : 기타지역에서의 출고량입니다.

1. **지역에 따라서 선호하는 게임 장르가 다를까 라는 질문에 대답을 하셔야합니다.**
2. **연도별 게임의 트렌드가 있을까 라는 질문에 대답을 하셔야합니다.**
3. **출고량이 높은 게임에 대한 분석 및 시각화 프로세스가 포함되어야 합니다. **


## **전처리**

---



In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16598 entries, 1 to 16598
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Name         16598 non-null  object 
 1   Platform     16598 non-null  object 
 2   Year         16327 non-null  float64
 3   Genre        16548 non-null  object 
 4   Publisher    16540 non-null  object 
 5   NA_Sales     16598 non-null  object 
 6   EU_Sales     16598 non-null  object 
 7   JP_Sales     16598 non-null  object 
 8   Other_Sales  16598 non-null  object 
dtypes: float64(1), object(8)
memory usage: 1.3+ MB


###**발견된 문제 **
1. `Platform` 에서 지원 플랫폼 명이 숫자인 데이터가 존재한다.
  
  -> 행을 모두 **제거한다.**
2. `Year` 에서 데이터가 N/A 인 데이터가 존재한다.
 
  -> 문자열인 듯 하였으나 다른 문제로 인해 문자열처럼 출력되는 현상이었다. **제거한다**.
3. `Year` 에서 연도숫자의 범위가 아닌 데이터가 존재한다. 
  
  -> 1950년 ~ 2050년 범위 이외의 데이터는 모두 **제거한다. **

  -> 자료형을 int로 교체한다.

4. `Genre, Publisher`가 공백인 데이터가 존재한다.

  -> 데이터를 모두 **제거한다.**

5. `Publisher` 이름이 숫자형인 데이터가 존재한다. 

  -> 이름이 숫자만 있는 행은 **제거한다.**

6. `Sales` 데이터들에서 K,M 등 값들의 단위가 중구난방이다.

  -> 모든 단위를 M 단위로 **통일한다**. 단위 표시가 없는 값은 모두 M으로 가정한다.

  -> 480K는 0.48 로 나타낸다. 4100K는 4.1 로 나타낸다.

  -> 숫자에 M,K가 붙은 경우 떼버린다.

  -> 자료형을 float로 교체한다.

7. 기종이 없이 밀려적혀 결측값이 존재하는 값이 존재한다.

  -> 제거한다.

**1.** Platform 에서 지원 플랫폼 명이 숫자인 데이터가 존재한다.

In [7]:
DF = df.copy() # 원본 데이터 수정 방지

In [8]:
NotPlatform = df[~df['Platform'].str.contains(r'[A-Za-z]')].index # 알파벳을 포함하지 않는 데이터 제거

In [9]:
DF = DF.drop(NotPlatform).reset_index(drop = True)

**2.** Year 에서 데이터가 N/A 인 데이터가 존재한다. 

**3.** 연도숫자의 범위가 아닌 데이터가 존재한다.

In [10]:
valid_Year = "Year>1950 & Year<2050"
DF = DF.query(valid_Year)

In [11]:
DF.Year = DF.Year.astype('int')  # int형으로 변경

**4.** 장르,퍼블리셔가 공백인 데이터가 존재한다.

In [12]:
DF.isnull().sum()

Name            0
Platform        0
Year            0
Genre          49
Publisher      36
NA_Sales        0
EU_Sales        0
JP_Sales        0
Other_Sales     0
dtype: int64

In [13]:
DF = DF.dropna()


In [14]:
DF = DF.reset_index()

**5.** `Publisher` 이름이 숫자형인 데이터가 존재한다. 

In [15]:
NotPub = DF[~DF['Publisher'].str.contains(r'[A-Za-z]')].index # 알파벳을 포함하지 않는 데이터 제거

In [16]:
NotPub # 없으니 pass 중간에 제거된듯

Int64Index([], dtype='int64')

In [17]:
DF.info() # 현재 상황

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16026 entries, 0 to 16025
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   index        16026 non-null  int64 
 1   Name         16026 non-null  object
 2   Platform     16026 non-null  object
 3   Year         16026 non-null  int64 
 4   Genre        16026 non-null  object
 5   Publisher    16026 non-null  object
 6   NA_Sales     16026 non-null  object
 7   EU_Sales     16026 non-null  object
 8   JP_Sales     16026 non-null  object
 9   Other_Sales  16026 non-null  object
dtypes: int64(2), object(8)
memory usage: 1.2+ MB


6.`Sales` 데이터들에서 K,M 등 값들의 단위가 중구난방이다.

-> 모든 단위를 M 단위로 통일한다.

-> 480K는 0.48 로 나타낸다. 4100K는 4.1 로 나타낸다.

-> 숫자에 M,K가 붙은 경우 떼버린다.

-> 자료형을 float로 교체한다.

In [39]:
def gett_value(DF,Sales):
  if DF[DF['Sales'].str.contains('K')]:
    DF = DF[DF['Sales'].str.replace('K','').astype('float')]
  return DF/1000

In [57]:
# file export
file_name = 'sample1.csv'
# saving the excelsheet
DF.to_csv(file_name, index=False)

In [130]:
uploaded = files.upload()

Saving sample3.csv to sample3.csv


In [131]:
DF = pd.read_csv('sample3.csv',index_col=0)

7. 결측치 제거

In [139]:
DF['Other_Sales'][DF['Other_Sales'].isnull()]

index
7531    NaN
16064   NaN
Name: Other_Sales, dtype: float64

In [140]:
DF = DF.dropna().reset_index(drop=True)

In [142]:
DF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16024 entries, 0 to 16023
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Name         16024 non-null  object 
 1   Platform     16024 non-null  object 
 2   Year         16024 non-null  object 
 3   Genre        16024 non-null  object 
 4   Publisher    16024 non-null  object 
 5   NA_Sales     16024 non-null  float64
 6   EU_Sales     16024 non-null  float64
 7   JP_Sales     16024 non-null  float64
 8   Other_Sales  16024 non-null  float64
dtypes: float64(4), object(5)
memory usage: 1.1+ MB


## **1. 지역에 따라서 선호하는 게임 장르**

### **1-1. Dot product**

- Dot product를 이용하여 구매자별 구매 총합을 계산하세요.
  - Data Description
    - People : 구매자
    - Name : 제품명
    - Price : 제품 개당 가격
    - Amount : 제품 구매 갯수

|People|Name|Price|Amount|
|:-:|:-:|:-:|:-:|
|X|A|100|35|
|X|B|500|10|
|X|C|250|25|
|X|D|50|40|
|Y|A|100|-|
|Y|B|500|50|
|Y|C|250|50|
|Y|D|50|-|
|Z|A|100|-|
|Z|B|500|-|
|Z|C|250|100|
|Z|D|50|-|

##### **Q1. Dot product를 통해 구한 구매자 X의 구매 총합을 쓰세요.**

In [ ]:
sp.Matrix(array) # array

Matrix([
[ 1,  0, 2],
[-3,  4, 6],
[-1, -2, 3]])